In [ ]:
!pip install diffusers accelerate transformers --quiet

In [ ]:
!pip install basicsr facexlib gfpgan --quiet

In [ ]:
from IPython.display import Image, clear_output, HTML, FileLink
import ipywidgets as widgets
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, DPMSolverMultistepScheduler
import torch
import requests
import random
import os
from PIL import Image as PILImage
from gfpgan import GFPGANer
import cv2
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url

In [ ]:
def generate_filename():
  random_number = random.randint(100000, 999999)
  filename = f"{random_number}.png"
  return filename

In [ ]:
def create_pipeline(model_name):
  pipe = StableDiffusionPipeline.from_pretrained(model_name, torch_dtype=torch.float16)
  pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
  pipe.safety_checker = None
  # diffusion_pipeline.enable_attention_slicing()
  # diffusion_pipeline.enable_vae_slicing()
  # diffusion_pipeline.enable_sequential_cpu_offload()
  # diffusion_pipeline.enable_vae_tiling()
  # diffusion_pipeline.enable_xformers_memory_efficient_attention()
  pipe.to("cuda")
  return pipe

In [ ]:
# function to generate images
def generate_image(pipe, prompt, negative_prompt, height, width, inference_steps, guidance_scale):
  print("generating image...")
  seed = random.randint(1,1000000)
  generator = torch.manual_seed(seed)
  image = pipe(prompt=prompt, generator=generator, negative_prompt=negative_prompt,height=height,width=width, num_inference_steps=inference_steps, guidance_scale=guidance_scale).images[0]
  filename = generate_filename();
  image.save(filename)
  return image, filename, seed

In [ ]:
# function to restore image
def restore_face(input_path, output_path):
  current_dir = os.getcwd()
  img_path = input_path
  output_path = output_path
  url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth'
  # model_path = "./GFPGANv1.3.pth"
  upscale = 1
  arch = "clean"
  channel_multiplier = 2
  bg_upsampler=None
  weight = 0.5
  aligned = None
  only_center_face = None

  restorer = GFPGANer(
      model_path=url,
      upscale=upscale,
      arch=arch,
      channel_multiplier=channel_multiplier,
      bg_upsampler=bg_upsampler
  )

  input_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
  # restore faces and background if necessary
  cropped_faces, restored_faces, restored_img = restorer.enhance(
      input_img,
      has_aligned=aligned,
      only_center_face=only_center_face,
      paste_back=True,
      weight=0.5)
  # print(restored_img)
  cv2.imwrite(output_path,restored_img)

In [ ]:
model_name = "XpucT/Deliberate"

In [ ]:
pipe = create_pipeline(model_name)

In [ ]:
prompt = "photo of a white lamborgini on road, city, cityscape, rain, cyberpunk"
# suffix = "professional, 8k, HDR, high detail, highly detailed, high resolution, hyper realistic, photo realistic, realistic, intricate, intricate sharp details, masterpiece, award winning, trending on artstation"
# negative_prompt = "deformed, distorted, disfigured, poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, mutated hands and fingers, disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation, low quality"
suffix = "professional, 8k, HDR, high detail, highly detailed, high resolution, hyper realistic, photo realistic, realistic, intricate, intricate sharp details, sony camera, mirrorless, 50mm"
negative_prompt = "deformed, distorted, disfigured, poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, mutated hands and fingers, disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation, low quality, low resolution"

height = 512
width = 512
inference_steps = 20
guidance_scale = 8

image, filename, seed = generate_image(pipe,prompt + ", " + suffix, negative_prompt, height, width, inference_steps, guidance_scale)

clear_output(wait=True)
display(image)
print(filename)
print(seed)

In [ ]:
restore_face(filename, "rest"+filename)
image = Image("rest"+filename)
display(image)

In [ ]:
import math

num_list = []

for i in range(512,2049,8):
  num_list.append(float(i))

# print(num_list)

limit_volume = 800 * 800
ratio = 16.0/9.0
max_vol = 0
h = 0
w = 0

for i in num_list:
  vol = i * (i*ratio)
  if(vol > max_vol and vol <= limit_volume):
    max_vol = vol
    h = i
    w = i * ratio

height_val = h
width_val = math.floor(w / 8) * 8
print(height_val)
print(width_val)
print(width_val/height_val)

In [ ]:
ratio = 16.0/9.0
width_num = 512
height_num = width_num * ratio
height_final = math.floor( height_num / 8) * 8
print(width_num)
print(height_final) 